In [1]:
import pandas as pd
import tqdm
import numpy as np
import codecs
import glob
import random
import matplotlib.pyplot as plt
import sys
#from wordcloud_functions import *
from scipy import stats

In [6]:

kaggle = pd.read_csv("kaggle_coefs.csv", sep='\t') # train set coefs

liar_split1 = pd.read_csv("liar_train_split1_coefs.csv", sep='\t')
liar_split2 = pd.read_csv("liar_train_split2_coefs.csv", sep='\t')

liar_random = pd.read_csv("liar_testanddev_random_labels.csv", sep='\t') #liar valid and test concatenated

liar2_random = pd.read_csv("liar_train_split2_random_labels_coefs.csv", sep='\t')
FakeNewsCorpus = pd.read_csv("FakeNewsCorpus_coefs.csv", sep='\t')

BS = pd.read_csv("BS_coefs.csv", sep='\t')

In [3]:
#classes = ['barely-true', 'false','half-true', 'mostly-true', 'pants-fire', 'true']
#liar_coefs.columns = [s.replace('_', ' ') for s in liar_coefs.columns]

#print(len(liar_coefs))
#print(len(classes))

#liar_coefs["class_labels"] = classes
#liar_coefs.set_index("class_labels", inplace=True)
#liar_coefs = liar_coefs.T
#false = liar_coefs.sort_values(by='false', ascending=False)
#false = false["false"]
#print(false.head(10))
#print(false.tail(10))

In [4]:
# find "best features" through highest coefficients of binary data

In [5]:
kaggle.columns = [s.replace('_', ' ') for s in kaggle.columns]
#kaggle_coefs.T.sort_values(by=0, ascending=False).tail(20)
kaggle.T.sort_values(by=0, ascending=False).head(30)

,0
anti,70.548679
october,66.038891
non,63.290661
year old,62.419340
2016,60.560375
co,55.820289
self,51.465532
november,48.626680
share,41.171381
us,37.830128


In [6]:
#liar.columns = [s.replace('_', ' ') for s in liar.columns]
#liar.T.sort_values(by=0, ascending=False).tail(20)
#liar.T.sort_values(by=0, ascending=False).head(30)

In [7]:
#liar_valid.columns = [s.replace('_', ' ') for s in liar_valid.columns]
#liar_valid.T.sort_values(by=0, ascending=False).tail(20)
#liar_valid.T.sort_values(by=0, ascending=False).head(30)

In [8]:
liar_random.columns = [s.replace('_', ' ') for s in liar_random.columns]
liar_random.T.sort_values(by=0, ascending=False).head(20)

,0
clinics,26.191684
payment,25.138101
president barack obama is,24.224021
benefits to,23.356478
away from,22.971684
his state,22.718489
bucks,22.511054
44,21.879811
34,21.774922
john mccain,21.764418


In [9]:
liar_split1.columns = [s.replace('_', ' ') for s in liar_split1.columns]
liar_split1.T.sort_values(by=0, ascending=False).head(10)

,0
is spending,60.505017
this is the,53.102194
holding,47.314712
few months,46.374002
patient,45.974196
impose,44.692535
of the population,44.451561
spending in,44.280928
anybody,44.122206
country that,43.574845


In [10]:
liar_split2.columns = [s.replace('_', ' ') for s in liar_split2.columns]
liar_split2.T.sort_values(by=0, ascending=False).head(10)

,0
million from,49.336907
there were,49.326025
president barack obama has,47.828363
and all,47.199551
arrested,46.356035
education funding,45.617625
texas public,45.387057
muslim,45.369580
of business,45.105594
38,44.713844


In [11]:
#FakeNewsNet.columns = [s.replace('_', ' ') for s in FakeNewsNet.columns]
#FakeNewsNet.T.sort_values(by=0, ascending=False).tail(20)
#FakeNewsNet.T.sort_values(by=0, ascending=False).head(5)#.index.tolist()


In [12]:
BS.columns = [s.replace('_', ' ') for s in BS.columns]
BS.T.sort_values(by=0, ascending=False).head(10)

,0
thats,154.071096
cant,103.936517
trumps,102.586082
theres,102.494565
hes,88.420775
wont,61.637292
peoples,61.586403
shes,59.208238
the worlds,57.603620
worlds,56.798442


In [12]:
# make common vocabulary (only shared words)
# Rank features med Mean reciprocal rank. Brug kun overlappende vocabulary mellem alle datasæt
# Prøv også Spearman rank order correlation:
# https://docs.scipy.org/doc/scipy-0.16.1/reference/generated/scipy.stats.spearmanr.html

#stats.spearmanr(a, b)

In [13]:
# 0 = True
# 1 = Fake

In [14]:
# LIAR (ene halvdel) er Ground truth, selvom den selvfølgelig ikke er sand på nogen måde. 
# Og så MRR for LIAR (anden del), random labels, og de andre fake news datasets.

#def mean_reciprocal_rank(rs):
#    rs = (np.asarray(r).nonzero()[0] for r in rs)
#    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

#from sklearn.metrics import label_ranking_average_precision_score
# Label ranking average precision
#If there is exactly one relevant label per sample, label ranking average precision is equivalent to the mean reciprocal rank.

In [44]:
from sklearn.metrics import label_ranking_average_precision_score
from sklearn import preprocessing

def top_features(df, n=50, head=True):
    df.columns = [s.replace('_', ' ') for s in df.columns]
    if head==False:
        features_sorted = df.T.sort_values(by=0, ascending=False).tail(n)
    else:
        features_sorted = df.T.sort_values(by=0, ascending=False).head(n)
    features_list = features_sorted.index.tolist()
    coefs = features_sorted[0].tolist()
    tuples = [(f, np.around(c, decimals=3)) for f,c in zip(features_list, coefs)]
    return tuples

def top_features_shared_vocab(df,n,vocab="liar1", head=True):
    if vocab=="liar1":
        liar_split1.columns = [s.replace('_', ' ') for s in liar_split1.columns]
        liar_feats = liar_split1.columns.tolist()
        common_feats = [i for i in df.columns if i in liar_feats]
    df = df[common_feats]
    return top_features(df,n=n,head=head)
    

def common_words(df):
    liar_split1.columns = [s.replace('_', ' ') for s in liar_split1.columns]
    liar_feats = liar_split1.columns.tolist()
    df.columns = [s.replace('_', ' ') for s in df.columns]
    common_feats = [i for i in df.columns if i in liar_feats]
    print("number of common words:",len(common_feats))
    
    ground_truth = liar_split1[common_feats].T.sort_values(by=0, ascending=False).index.tolist()
    other = df[common_feats].T.sort_values(by=0, ascending=False).index.tolist()
    
    ground_truth_df = liar_split1[common_feats].T.sort_values(by=0, ascending=False)
    other_df = df[common_feats].T.sort_values(by=0, ascending=False)
    
    #ground_truth_coefs = liar_split1[common_feats].T.sort_values(by=0, ascending=False).values.tolist()
    #other_coefs = df[common_feats].T.sort_values(by=0, ascending=False).values.tolist()
    #print(other_coefs)
    
    # list coefficients of common words after liar's coeff order (ground truth)
    return ground_truth, other, common_feats, ground_truth_df, other_df#, ground_truth_coefs, other_coefs


def sort_other_by_ground_index(ground_truth_df, other_df):
    df1 = other_df
    df1 = df1.reindex(index=ground_truth_df.index)
    return df1



def MRR(df):
    ground_truth, other, common_feats, ground_truth_df, other_df = common_words(df)
    
    lb = preprocessing.LabelBinarizer()
    ground_truth = lb.fit_transform(ground_truth)
    other = lb.transform(other)
    mrr = label_ranking_average_precision_score(ground_truth, other)

    df1 = other_df
    df1 = df1.reindex(index=ground_truth_df.index)
    df_both = pd.concat([ground_truth_df, df1], axis=1)
    df_both.columns = ["Liar_split1_coefs", "Liar_split2_coefs"]
    #print(stats.spearmanr(df_both["Liar_split1_coefs"],df_both["Liar_split2_coefs"] ))
    spear = stats.spearmanr(df_both["Liar_split1_coefs"],df_both["Liar_split2_coefs"])
    kendall = stats.kendalltau(df_both["Liar_split1_coefs"],df_both["Liar_split2_coefs"])
    
    return kendall, spear, mrr


In [8]:
ground_truth, other, common_feats, ground_truth_df, other_df = common_words(liar2_random)

number of common words: 3567


In [17]:
df1 = other_df
df1 = df1.reindex(index=ground_truth_df.index)
df_both = pd.concat([ground_truth_df, df1], axis=1)
df_both.columns = ["Liar_split1_coefs", "Liar_split2_coefs"]
print(stats.spearmanr(df_both["Liar_split1_coefs"],df_both["Liar_split2_coefs"] ))
df_both

SpearmanrResult(correlation=-0.001346361813274133, pvalue=0.938579796711495)


,Liar_split1_coefs,Liar_split2_coefs
holding,47.314712,8.040086
impose,44.692535,3.055944
spending in,44.280928,5.166034
country that,43.574845,-5.107778
says charlie,42.999183,-12.597770
governments,41.392231,-0.547788
per year,41.215893,5.642665
of the states,40.829049,-2.619902
was in,40.555162,14.414202
over half,39.712781,-7.523245


In [18]:
#ground_truth

In [7]:
ls = top_features(BS, n=20, head=True)
print(ls)

[('thats', 154.071), ('cant', 103.937), ('trumps', 102.586), ('theres', 102.495), ('hes', 88.421), ('wont', 61.637), ('peoples', 61.586), ('shes', 59.208), ('the worlds', 57.604), ('worlds', 56.798), ('lets', 53.148), ('dont', 52.507), ('whats', 51.985), ('id', 51.66), ('ten', 49.777), ('stated', 49.774), ('its not', 48.866), ('todays', 44.888), ('proven', 44.678), ('heres', 40.47)]


In [48]:
sls = top_features_shared_vocab(FakeNewsCorpus, n=20, head=True)
print(sls)

[('next', 38.065), ('2016', 34.329), ('source', 32.168), ('news', 27.957), ('per', 25.101), ('figure', 24.093), ('else', 23.092), ('trump', 21.259), ('above', 20.758), ('illegal', 20.263), ('under', 19.339), ('this', 19.046), ('university of', 18.976), ('2015', 18.804), ('century', 18.407), ('says', 17.967), ('terrorists', 17.383), ('similar', 16.391), ('keep', 16.178), ('member', 15.912)]


In [14]:
print("liar split 2")
print(MRR(liar_split2))
print()
print("liar random")
print(MRR(liar_random))
print()
print("liar split 2 random")
print(MRR(liar2_random))
print()
print("kaggle")
print(MRR(kaggle))
print()
print("FakeNewsCorpus")
print(MRR(FakeNewsCorpus))
print()
print("BS")
print(MRR(BS))

liar split 2
number of common words: 3567
(KendalltauResult(correlation=0.023055867040466062, pvalue=0.038977851315390756), SpearmanrResult(correlation=0.033932012021838334, pvalue=0.042719597409482224), 0.0002803476310625285)

liar random
number of common words: 3278
(KendalltauResult(correlation=-0.000949585836306722, pvalue=0.935041169839104), SpearmanrResult(correlation=-0.001346361813274133, pvalue=0.938579796711495), 0.0006100350628239306)

liar split 2 random
number of common words: 3567
(KendalltauResult(correlation=-0.003844793431586143, pvalue=0.7306488674451868), SpearmanrResult(correlation=-0.005363772931958838, pvalue=0.748787944068928), 0.0005606166673307913)

kaggle
number of common words: 2462
(KendalltauResult(correlation=-0.00034824613509729893, pvalue=0.9793366379128449), SpearmanrResult(correlation=-0.0005693181577948438, pvalue=0.9774752047893994), 0.00040617384240457276)

FakeNewsCorpus
number of common words: 2362
(KendalltauResult(correlation=0.00559907542830200